### Tiền xử lý chung cho dữ liệu

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

data_path = os.path.join('..', 'data', 'raw', 'netflix_titles.csv')
df = pd.read_csv(data_path)

# loại bỏ cột `show_id`
df = df.drop(columns=['show_id'])
# Chuyển cột `duration` sang số (lấy phần số)
df['duration'] = df['duration'].astype(str).str.extract('([0-9]+)')
df['duration'] = df['duration'].astype('Int64')
# Chuẩn hóa `date_added` (loại bỏ format không hợp lệ)
df['date_added'] = pd.to_datetime(df['date_added'].str.strip(), errors='coerce')

# Xử lý dữ liệu bị thiếu
# 1. Điền giá trị 'Unknown' cho các biến phân loại (Categorical)
cols_to_fill = ['director', 'cast', 'country']
for col in cols_to_fill:
	df[col].fillna('Unknown', inplace=True)
	print(f"-> Đã điền 'Unknown' cho cột: {col}")

# 2. Xóa các dòng thiếu dữ liệu ở các cột quan trọng khác
cols_to_drop = ['date_added', 'rating', 'duration']
before_rows = df.shape[0]
df.dropna(subset=cols_to_drop, inplace=True)
after_rows = df.shape[0]
print(f"-> Đã xóa {before_rows - after_rows} dòng thiếu dữ liệu ở {cols_to_drop}.")

# Kiểm tra lại lần cuối
print(f"\nKiểm tra lại dữ liệu thiếu: {df.isnull().sum().sum()} giá trị null còn lại.")

-> Đã điền 'Unknown' cho cột: director
-> Đã điền 'Unknown' cho cột: cast
-> Đã điền 'Unknown' cho cột: country
-> Đã xóa 17 dòng thiếu dữ liệu ở ['date_added', 'rating', 'duration'].

Kiểm tra lại dữ liệu thiếu: 0 giá trị null còn lại.


C:\Users\ACER\AppData\Local\Temp\ipykernel_28140\1460215892.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Unknown', inplace=True)


In [2]:
# Lưu kết quả đã tiền xử lý vào thư mục data/processed (cả parquet và csv).
import os
out_dir = os.path.join('..', 'data', 'processed')
os.makedirs(out_dir, exist_ok=True)
# Chọn dataframe đã xử lý phù hợp theo thứ tự ưu tiên
df_to_save = locals().get('df_double_exploded', locals().get('df_exploded', locals().get('df_clean', locals().get('df'))))
if df_to_save is None:
    raise RuntimeError('Không tìm thấy dataframe đã xử lý để lưu. Hãy chạy các ô trước đó để tạo dataframe.')
parquet_path = os.path.join(out_dir, 'netflix_cleaned.parquet')
csv_path = os.path.join(out_dir, 'netflix_cleaned.csv')
print(f'Saving processed dataset to: {parquet_path} and {csv_path}')
df_to_save.to_parquet(parquet_path, index=False)
df_to_save.to_csv(csv_path, index=False)
print('Saved processed dataset.')

Saving processed dataset to: ..\data\processed\netflix_cleaned.parquet and ..\data\processed\netflix_cleaned.csv
Saved processed dataset.
